# metacognitive prompting

### import statements

In [25]:
from openai import OpenAI
from dotenv import load_dotenv
from ipywidgets import widgets, Output
from IPython.display import display
import os
import threading

In [26]:
load_dotenv()
text_input=""
chat_history = {
    
}
chat_output=Output()
active = True
processing_message = threading.Event()

In [27]:
apikey = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=apikey
)
print(client)

In [28]:
def new_chat():
    chat_history.clear()

:::{admonition} `stream_chatbot` docs
:class: dropdown 

### `stream_chatbot(message)`

This function interacts with the chatbot using stream functionality.

**Parameters:**
- `message`: The message sent by the user to the chatbot.

**Returns:**
- None

**Functionality:**
- Sends the user message to the chatbot.
- Receives and prints the response from the chatbot.
```

In [29]:
def stream_chatbot(message):
    global chat_output
    chat_completion_stream = client.chat.completions.create(
        messages=[
            {
                "role":"user",
                "content":message,
             },
        ],
        model="gpt-3.5-turbo",
        stream=True
    )
    print(chat_completion_stream)
    for chunk in chat_completion_stream:
        if chunk.choices[0].delta.content is not None:
            chat_output.append_stdout(f"{chunk.choices[0].delta.content}")
    chat_output.append_stdout(f"\n")

:::{admonition} `prompt_user` docs
:class: dropdown

### `prompt_user()`

This function prompts the user for input.

**Parameters:**
- None

**Returns:**
- `user_input`: The input provided by the user.

**Functionality:**
- Displays a prompt for the user to input a message.
- Returns the user's input.

In [30]:
def prompt_user():
    user_input = input("You: ")
    return user_input

:::{admonition} `chatbot` docs
:class: dropdown

### `chatbot(user_message)`

This function interacts with the chatbot based on user input.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Initiates interaction with the chatbot by passing the user's message.
- Prints the chatbots response.


In [31]:
def chatbot(user_message):
    global active
    global chat_output
    processing_message.set()
    chat_output.append_stdout("Chatbot: ")
    if user_message.lower() in ["bye!", "quit", "exit"]:
        chat_output.append_stdout("BYE\n")
        active = False
    else:
        stream_chatbot(user_message)
    processing_message.clear()

:::{admonition} `print_user_message` docs
:class: dropdown

### `print_user_message(user_message)`

This function prints the user's message.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Prints the user's message in the format: "You: [user_message]".


In [32]:
def print_user_message(user_message):
    global chat_output
    chat_output.append_stdout(f"You: {user_message}\n")

In [33]:
def handle_user_message(user_message):
    print_user_message(user_message)
    chatbot(user_message)

In [34]:
def on_submit_button_clicked(b):
    if not processing_message.is_set():
        handle_user_message(text_input.value)
        text_input.value=""

In [35]:
def chat_interface():
    global text_input
    text_input = widgets.Text(description="Your message: ")
    submit_button = widgets.Button(description="Submit", disabled=False)
    
    submit_button.on_click(on_submit_button_clicked)    
    
    display(chat_output, text_input, submit_button)

In [36]:
def main():
    new_chat()
    chat_interface()

if __name__ == "__main__": main()

Output()

Text(value='', description='Your message: ')

Button(description='Submit', style=ButtonStyle())